## Exercise 7 - Final

This final exercise combines different parts from the previous exercises.

### Section 1

1 Search Digitraffic's REST interface service (in JSON format) for active trains with the terminal address `/live-trains` from Helsinki station with the following search parameters:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False

In [1]:
import requests

station = 'HKI' # Helsinki 

arrived_trains = 10
departed_trains = 10
include_nonstopping = False

params = {
    'arrived_trains': arrived_trains,
    'departed_trains': departed_trains,
    'include_nonstopping': include_nonstopping
}

url = f"https://rata.digitraffic.fi/api/v1/live-trains/station/{station}/"

# https://rata.digitraffic.fi/api/v1/live-trains/station/HKI?version=0&arrived_trains=10&arriving_trains=5&departed_trains=10&departing_trains=5&include_nonstopping=false
req = requests.get(url, params)
req.status_code

200

2 Read the data into the Pandas dataframe without editing, so that the column headings are as follows:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version`

In [2]:
import pandas as pd
df = pd.json_normalize(req.json())

df.head()

,trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
0,27,2024-02-19,10,vr,IC,Long-distance,,True,False,287676056112,REGULAR,2023-12-14T06:21:07.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
1,957,2024-02-19,10,vr,S,Long-distance,,True,False,287676043618,REGULAR,2023-12-14T06:21:07.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
2,8118,2024-02-19,10,vr,HL,Commuter,A,False,False,287676008287,REGULAR,2023-11-02T05:57:22.000Z,"[{'stationShortCode': 'LPV', 'stationUICCode':..."
3,8125,2024-02-19,10,vr,HL,Commuter,A,True,False,287676054003,REGULAR,2023-11-02T05:57:22.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
4,8127,2024-02-19,10,vr,HL,Commuter,A,False,False,287676055540,REGULAR,2023-11-02T05:57:22.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."


3 Delete all other columns except `timeTableRows` and `trainNumber`.

In [3]:
df = df.loc[:, ['timeTableRows', 'trainNumber']]
df.head()

,timeTableRows,trainNumber
0,"[{'stationShortCode': 'HKI', 'stationUICCode':...",27
1,"[{'stationShortCode': 'HKI', 'stationUICCode':...",957
2,"[{'stationShortCode': 'LPV', 'stationUICCode':...",8118
3,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8125
4,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8127


4 After this, extract the values ​​of the `timeTableRows` column from each train separately (the column value for each train contains several JSON objects) and add `trainNumber`, which identifies the train data.

In [4]:

# Extract data from nested JSON
new_data = []

for index, row in df.iterrows(): # loop over every row
    train_number = row['trainNumber']

    # loop over entries in timeTableRows[]
    for entry in row['timeTableRows']: # entry => one data row
        new_data_row = {}   
        new_data_row['trainNumber'] = row['trainNumber']
        for key, value in entry.items():
            new_data_row[key] = value
    
        new_data.append(new_data_row)

# Create a new DataFrame based on the extracted rows
# new_dataframe = pd.json_normalize(new_data)
new_dataframe = pd.DataFrame(new_data)
new_dataframe.head()


,trainNumber,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialStop,commercialTrack,cancelled,scheduledTime,liveEstimateTime,estimateSource,actualTime,differenceInMinutes,causes,trainReady
0,27,HKI,1,FI,DEPARTURE,True,True,9,False,2024-02-19T12:24:00.000Z,2024-02-19T12:45:00.000Z,LIIKE_USER,2024-02-19T12:44:31.000Z,21.0,[],"{'source': 'KUPLA', 'accepted': True, 'timesta..."
1,27,PSL,10,FI,ARRIVAL,True,True,3,False,2024-02-19T12:29:00.000Z,2024-02-19T12:51:27.000Z,LIIKE_AUTOMATIC,NaN,22.0,[],NaN
2,27,PSL,10,FI,DEPARTURE,True,True,3,False,2024-02-19T12:30:00.000Z,2024-02-19T12:52:27.000Z,LIIKE_AUTOMATIC,NaN,22.0,[],NaN
3,27,KÄP,977,FI,ARRIVAL,False,NaN,,False,2024-02-19T12:32:00.000Z,2024-02-19T12:54:20.000Z,LIIKE_AUTOMATIC,NaN,22.0,[],NaN
4,27,KÄP,977,FI,DEPARTURE,False,NaN,,False,2024-02-19T12:32:00.000Z,2024-02-19T12:54:20.000Z,LIIKE_AUTOMATIC,NaN,22.0,[],NaN


5 Then drop the following columns from the data: `causes`, `differenceInMinutes` and `commercialStop`.

In [5]:
df = new_dataframe.drop(['causes', 'differenceInMinutes', 'commercialStop'], axis=1)

df.head()

# df.head()

,trainNumber,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,liveEstimateTime,estimateSource,actualTime,trainReady
0,27,HKI,1,FI,DEPARTURE,True,9,False,2024-02-19T12:24:00.000Z,2024-02-19T12:45:00.000Z,LIIKE_USER,2024-02-19T12:44:31.000Z,"{'source': 'KUPLA', 'accepted': True, 'timesta..."
1,27,PSL,10,FI,ARRIVAL,True,3,False,2024-02-19T12:29:00.000Z,2024-02-19T12:51:27.000Z,LIIKE_AUTOMATIC,NaN,NaN
2,27,PSL,10,FI,DEPARTURE,True,3,False,2024-02-19T12:30:00.000Z,2024-02-19T12:52:27.000Z,LIIKE_AUTOMATIC,NaN,NaN
3,27,KÄP,977,FI,ARRIVAL,False,,False,2024-02-19T12:32:00.000Z,2024-02-19T12:54:20.000Z,LIIKE_AUTOMATIC,NaN,NaN
4,27,KÄP,977,FI,DEPARTURE,False,,False,2024-02-19T12:32:00.000Z,2024-02-19T12:54:20.000Z,LIIKE_AUTOMATIC,NaN,NaN


6 Fill in the missing values ​​of the `actualTime` column in each row with the values of the `scheduledTime` column.

In [6]:
df['actualTime'] = df['actualTime'].fillna(df['scheduledTime'])

df.head()

,trainNumber,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,liveEstimateTime,estimateSource,actualTime,trainReady
0,27,HKI,1,FI,DEPARTURE,True,9,False,2024-02-19T12:24:00.000Z,2024-02-19T12:45:00.000Z,LIIKE_USER,2024-02-19T12:44:31.000Z,"{'source': 'KUPLA', 'accepted': True, 'timesta..."
1,27,PSL,10,FI,ARRIVAL,True,3,False,2024-02-19T12:29:00.000Z,2024-02-19T12:51:27.000Z,LIIKE_AUTOMATIC,2024-02-19T12:29:00.000Z,NaN
2,27,PSL,10,FI,DEPARTURE,True,3,False,2024-02-19T12:30:00.000Z,2024-02-19T12:52:27.000Z,LIIKE_AUTOMATIC,2024-02-19T12:30:00.000Z,NaN
3,27,KÄP,977,FI,ARRIVAL,False,,False,2024-02-19T12:32:00.000Z,2024-02-19T12:54:20.000Z,LIIKE_AUTOMATIC,2024-02-19T12:32:00.000Z,NaN
4,27,KÄP,977,FI,DEPARTURE,False,,False,2024-02-19T12:32:00.000Z,2024-02-19T12:54:20.000Z,LIIKE_AUTOMATIC,2024-02-19T12:32:00.000Z,NaN


### Section 2

7 Extract the cell values ​​of the `table` element of the `data.html` file using the regex syntax and connect them to the dataframe created in step 1. **Note**: only regex should be used in the task (E.g. the use of Pandas' read_html() method is prohibited)!

In [7]:
import re

with open("../exercises/data.html") as file:
    html_data = file.read()
    # remove all white space, line breaks and tabs
    html_data = html_data.replace(' ', '').replace('\n', '').replace('\t', '')

# tr_pattern = r"<tr[^>]*>(.*?)<\/tr>" # table rows, ignore attributes
th_pattern = r"<th>(.*?)<\/th>" # dataframe columns
td_pattern = r"<td>(.*?)<\/td>" # dataframe rows

header = re.findall(th_pattern, html_data)
cells = re.findall(td_pattern, html_data)

# there are 10 columns
# split the list containing cell values at every 10 steps
# so we have one list as one row
cell_rows = [] # list of lists, each nested list represents values for one row
chunk_size = len(header)

for i in range(0, len(cells), chunk_size):
    cell_rows.append(cells[i:i + chunk_size])

data = []  # list of dicts [{}, {},{}]
for row in cell_rows:
    entry = {} # 3 rows = 3 entries
    for index, vals in enumerate(row):
        entry[header[index]]  = row[index] 
    data.append(entry)

new_df = pd.json_normalize(data)

df = pd.concat([df, new_df]).reset_index(drop=True)
df.tail()


,trainNumber,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,liveEstimateTime,estimateSource,actualTime,trainReady
1484,64126,PSL,10,FI,DEPARTURE,False,7,False,2024-02-19T12:35:00.000Z,NaN,NaN,2024-02-19T12:25:59.000Z,NaN
1485,64126,HKI,1,FI,ARRIVAL,True,11,False,2024-02-19T12:40:00.000Z,NaN,NaN,2024-02-19T12:45:43.000Z,NaN
1486,150,JVS,1272,FI,DEPARTURE,False,,False,2019-08-06T04:48:00.000Z,NaN,NaN,2019-08-06T04:48:00.000Z,NaN
1487,150,R702,98998,FI,ARRIVAL,False,,False,2019-08-06T06:04:39.000Z,NaN,NaN,2019-08-06T06:04:39.000Z,NaN
1488,8337,ILA,98998,FI,ARRIVAL,9,,False,2019-08-06T07:03:12.000Z,NaN,NaN,2019-08-06T07:03:12.000Z,NaN


### Section 3

8 Make a SQLite database and insert the dataframe obtained as a result of step 2 to the database.

In [8]:
import sqlite3

# Create a database connection
db_connection = sqlite3.connect('final.db')

# convert all values to str
# get rid of error: ProgrammingError: Error binding parameter 11: type 'dict' is not supported
df = df.applymap(str)

# Save dataframe to database
df.to_sql(name='live_trains', con=db_connection, if_exists='fail', index=False)

# Close the connect
# Close the connect
db_connection.commit()
db_connection.close()